In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch 
from pinns_phase import Pinns, PinnLaterPhase
import matplotlib.pyplot as plt


In [ ]:
kwargs = {
    "alpha_f" : 0.005,
    "h_f" : 5,
    "T_hot" : 4,
    "T0" : 1,
    "T_cold" : 1,
}

n_int = 128
n_sb = 64
n_tb = 64

pins_phases = []
hists_phases = []

In [ ]:
pinn = Pinns(n_int, n_sb, n_tb, 0, 1, **kwargs)
hist = pinn.fit(num_epochs=1, max_iter=500, verbose=True)

pins_phases.append(pinn)
hists_phases.append(hist)

In [ ]:
pinn.plot()

In [ ]:
pinn.

In [ ]:
for i in range(1, 4):
    t0 = i
    tf = i+1

    old_pinn = pins_phases[i-1]
    new_pinn = PinnLaterPhase(n_int, n_sb, n_tb, t0, tf, old_pinn, **kwargs)
    hist = new_pinn.fit(num_epochs=1, max_iter=5000, verbose=True)
    pins_phases.append(new_pinn)
    # torch.save(new_pinn.approximate_solution.state_dict(), f'saved_models/pinn_{t0}.pth')


In [ ]:
for i in range(4, 6):
    t0 = i
    tf = i+1

    old_pinn = pins_phases[i-1]
    new_pinn = PinnLaterPhase(n_int, n_sb, n_tb, t0, tf, old_pinn, **kwargs)
    print(f'##############################  Phase {t0}  ##############################')
    hist = new_pinn.fit(num_epochs=1, max_iter=5000, verbose=True)
    pins_phases.append(new_pinn)
    hists_phases.append(hist)
    torch.save(new_pinn.approximate_solution.state_dict(), f'saved_models/pinn_{t0}.pth')


In [ ]:
outputs_tfs = []
outputs_tss = []
inputs_ = []
for pinn in pins_phases:
    inputs = pinn.convert(pinn.soboleng.draw(5000))
    output_tf = pinn.approximate_solution(inputs)
    output_ts = pinn.approximate_coefficient(inputs)

    inputs_.append(inputs)
    outputs_tfs.append(output_tf)
    outputs_tss.append(output_ts)

inputs = torch.cat(inputs_)
outputs_tf = torch.cat(outputs_tfs)
outputs_ts = torch.cat(outputs_tss)

output = torch.cat([outputs_tf, outputs_ts], 1)

In [ ]:

labels = ["T_f", "T_s"]
_, axs = plt.subplots(1, 2, figsize=(16, 6), dpi=100)
# lims = [(1, 4), (1, 3)]
for i in range(2):
    im = axs[i].scatter(
        inputs[:, 0].detach(),
        inputs[:, 1].detach(),
        c=output[:, i].detach(),
        cmap="jet",
        # clim=lims[i]
    )
    axs[i].set_xlabel("t")
    axs[i].set_ylabel("x")
    axs[i].grid(True, which="both", ls=":")
    axs[i].set_title(f"Approximate Solution {labels[i]}")
    plt.colorbar(im, ax=axs[i])
plt.show()

In [ ]:
pins_phases[4].plot_reference()

In [ ]:
pins_phases[4].plot()